
## Passwords Auditor with contraints


In [1]:

## !pip install zxcvbn
## !pip install password-strength
## !pip install passlib




In [2]:


import string
import torch
import torch.nn as nn
import torch.nn.functional as F

from zxcvbn import zxcvbn


In [3]:


VOCAB = list(string.ascii_letters + string.digits + "!@#$%^&*")
V     = len(VOCAB)
V


70

In [4]:

MASK_UPPER = torch.tensor([c in string.ascii_uppercase for c in VOCAB]).float()
MASK_LOWER = torch.tensor([c in string.ascii_lowercase for c in VOCAB]).float()
MASK_DIGIT = torch.tensor([c in string.digits          for c in VOCAB]).float()
MASK_SPEC  = torch.tensor([c in "!@#$%^&*"             for c in VOCAB]).float()

print(MASK_SPEC.shape)

MASK_SPEC


torch.Size([70])


tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1.])

In [5]:


class TinyDecoder3(nn.Module):
    
    def __init__(self, z_dim=128, T=12, V=V):
        super().__init__()
        self.T = T
        self.V = V
        
        self.net = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.ReLU(),
            nn.Linear(256, T * V)
        )

    def forward(self, z):
        logits = self.net(z)                  # [B, T*V]
        
        return logits.view(z.size(0), self.T, self.V)  # [B, T, V]



In [6]:

from password_strength import PasswordStats

def score_with_password_strength(passwords):
    stats = [PasswordStats(pw) for pw in passwords]
    return {
        "average_strength": sum(s.strength() for s in stats) / len(stats),
        "weak%": sum(s.strength() < 0.3 for s in stats) / len(stats) * 100,
        "strong%": sum(s.strength() > 0.7 for s in stats) / len(stats) * 100
    }



In [7]:

import math

def estimate_entropy(passwd):
    charset_size = 0
    if any(c.islower() for c in passwd): charset_size += 26
    if any(c.isupper() for c in passwd): charset_size += 26
    if any(c.isdigit() for c in passwd): charset_size += 10
    if any(c in "!@#$%^&*()-_=+[{]};:'\",<.>/?\\|" for c in passwd): charset_size += 32
    if charset_size == 0: charset_size = 1
    return len(passwd) * math.log2(charset_size)

def average_entropy(passwords):
    return sum(estimate_entropy(p) for p in passwords) / len(passwords)



In [8]:

import math

def estimate_entropy(password):
    charset_size = 0
    if any(c.islower() for c in password): charset_size += 26
    if any(c.isupper() for c in password): charset_size += 26
    if any(c.isdigit() for c in password): charset_size += 10
    if any(c in "!@#$%^&*()-_=+[]{};:'\",.<>/?\\|" for c in password): charset_size += 32
    return len(password) * math.log2(charset_size or 1)

def entropy_stats(passwords):
    entropies = [estimate_entropy(pw) for pw in passwords]
    return {
        "avg_entropy_bits": sum(entropies) / len(entropies),
        "min": min(entropies),
        "max": max(entropies)
    }



In [9]:

class TinyDecoder(nn.Module):
    def __init__(self, z_dim=128, T=12, V=V):
        super().__init__()
        self.T = T
        self.V = V
        self.linear = nn.Linear(z_dim, T * V)

    def forward(self, z):
        logits = self.linear(z)                      # [B, T*V]
        return logits.view(z.size(0), self.T, self.V)  # [B, T, V]



In [10]:


def st_gumbel_softmax(logits, tau):
    
    eps = 1e-9
    
    g = -torch.log(-torch.log(torch.rand_like(logits) + eps) + eps)
    
    y_soft = F.softmax((logits + g) / tau, dim=-1)          # [B, T, V]
    
    y_hard = F.one_hot(
                 y_soft.argmax(-1), 
                 y_soft.size(-1)
    ).float()
    
    return y_hard + (y_soft - y_soft.detach())              # hard forward, soft grad


In [11]:


def class_present(p_pos):
    
    # p_pos: [B, T] probability per position of being in the class
    
    return 1.0 - torch.prod(1.0 - p_pos + 1e-6, dim=1)      # [B]



In [12]:


def entropy_bits_per_string(y_probs):
    # y_probs: [B, T, V], probs per token
    p = y_probs.clamp_min(1e-9)
    H_t = -(p * p.log()).sum(dim=-1)                         # nats, [B, T]
    H = H_t.mean(dim=1)                                      # [B]
    return H / torch.log(torch.tensor(2.0))                  # bits


In [13]:

# --- Decode one-hot/probabilities to strings ---

def decode(y_probs):
    idx = y_probs.argmax(dim=-1).cpu()                       # [B, T]
    out = []
    for row in idx.tolist():
        out.append("".join(VOCAB[i] for i in row))
    return out


In [14]:

def generate_passwords(batch=16, steps=300, T=12, H_min_bits=3.0, seed=0):
    torch.manual_seed(seed)

    model = TinyDecoder(T=T)
    for p in model.parameters():
        p.requires_grad = False  # decoder is fixed

    # ⚙️ Weak seeds
    weak = [
        "password", "qwerty", "letmein", "admin",
        "welcome", "123456", "iloveyou", "guest",
        "hello123", "abc123", "monkey", "test",
        "summer", "dragon", "football", "name"
    ][:batch]
    weak = [pw[:T].ljust(T, "a") for pw in weak]

    # 🔤 One-hot-style logit init
    z_init = torch.full((batch, T, V), -6.0)
    for b, pw in enumerate(weak):
        for t, c in enumerate(pw):
            if c in VOCAB:
                z_init[b, t, VOCAB.index(c)] = 6.0

    z = z_init.clone().detach().requires_grad_(True)
    opt = torch.optim.Adam([z], lr=0.05)

    for step in range(steps):
        tau = max(0.8 - 0.003 * step, 0.2)
        y_probs = F.softmax(z / tau, dim=-1)

        # 🔍 Check class coverage
        pU = (y_probs * MASK_UPPER).sum(dim=-1)
        pL = (y_probs * MASK_LOWER).sum(dim=-1)
        pD = (y_probs * MASK_DIGIT).sum(dim=-1)
        pS = (y_probs * MASK_SPEC ).sum(dim=-1)

        class_loss = (
            0*F.relu(1.0 - class_present(pU)).mean() +
            F.relu(1.0 - class_present(pL)).mean() +
            F.relu(1.0 - class_present(pD)).mean() +
            F.relu(1.0 - class_present(pS)).mean()
        )

        # 📈 Entropy enforcement
        H_bits = entropy_bits_per_string(y_probs)
        entropy_loss = F.relu(H_min_bits - H_bits).mean()

        # 🎯 Final loss — no divergence penalty
        loss = class_loss + entropy_loss

        opt.zero_grad()
        loss.backward()
        opt.step()

    return decode(y_probs.detach())



In [15]:

def generate_passwords_maybe(batch=16, steps=200, T=12, H_min_bits=3.0, seed=0):
    torch.manual_seed(seed)

    model = TinyDecoder(T=T)
    for p in model.parameters():
        p.requires_grad = False  # freeze decoder

    # ✅ Define weak initial passwords
    weak = [
        "password", "qwerty", "letmein", "admin",
        "welcome", "123456", "iloveyou", "guest",
        "hello123", "abc123", "monkey", "test",
        "summer", "dragon", "football", "name"
    ][:batch]
    weak = [pw[:T].ljust(T, "a") for pw in weak]

    # ✅ Create z_init based on those weak passwords
    z_init = torch.full((batch, T, V), -6.0)  # low everywhere
    for b, pw in enumerate(weak):
        for t, c in enumerate(pw):
            if c in VOCAB:
                z_init[b, t, VOCAB.index(c)] = 6.0  # high where char matches

    z = z_init.clone().detach().requires_grad_(True)

    opt = torch.optim.Adam([z], lr=0.05)
    y_probs = None

    for t in range(steps):
        tau = max(0.8 - 0.003 * t, 0.2)
        logits = z
        y_probs = F.softmax(logits / tau, dim=-1)  # <-- replaced Gumbel

        # Char class presence
        pU = (y_probs * MASK_UPPER).sum(dim=-1)
        pL = (y_probs * MASK_LOWER).sum(dim=-1)
        pD = (y_probs * MASK_DIGIT).sum(dim=-1)
        pS = (y_probs * MASK_SPEC ).sum(dim=-1)

        need_upper = F.relu(1.0 - class_present(pU)).mean()
        need_lower = F.relu(1.0 - class_present(pL)).mean()
        need_digit = F.relu(1.0 - class_present(pD)).mean()
        need_spec  = F.relu(1.0 - class_present(pS)).mean()
        class_loss = need_upper + need_lower + need_digit + need_spec

        init_soft = F.softmax(z_init / tau, dim=-1).detach()
        change_penalty = (y_probs - init_soft).abs().sum() / batch

        # Optional entropy constraint
        H_bits = entropy_bits_per_string(y_probs)
        entropy_loss = F.relu(H_min_bits - H_bits).mean()

        # Final loss
        loss = class_loss + entropy_loss ## + 0.1 * change_penalty

        opt.zero_grad()
        loss.backward()
        opt.step()

    return decode(y_probs.detach())



In [16]:

def generate_passwords800(batch=16, steps=200, T=12, H_min_bits=3.0, seed=0):
    torch.manual_seed(seed)

    model = TinyDecoder(T=T)
    for p in model.parameters():
        p.requires_grad = False  # freeze decoder

    # ✅ Define weak initial passwords
    weak = [
        "password", "qwerty", "letmein", "admin",
        "welcome", "123456", "iloveyou", "guest",
        "hello123", "abc123", "monkey", "test",
        "summer", "dragon", "football", "name"
    ][:batch]
    weak = [pw[:T].ljust(T, "a") for pw in weak]

    # ✅ Create z_init based on those weak passwords
    z_init = torch.full((batch, T, V), -6.0)  # low everywhere
    for b, pw in enumerate(weak):
        for t, c in enumerate(pw):
            if c in VOCAB:
                z_init[b, t, VOCAB.index(c)] = 6.0  # high where char matches

    z = z_init.clone().detach().requires_grad_(True)

    opt = torch.optim.Adam([z], lr=0.05)
    y_probs = None

    for t in range(steps):
        tau = max(0.8 - 0.003 * t, 0.2)
        logits = z
        y_probs = st_gumbel_softmax(logits, tau)  # [B, T, V]

        # Char class presence
        pU = (y_probs * MASK_UPPER).sum(dim=-1)
        pL = (y_probs * MASK_LOWER).sum(dim=-1)
        pD = (y_probs * MASK_DIGIT).sum(dim=-1)
        pS = (y_probs * MASK_SPEC ).sum(dim=-1)

        need_upper = F.relu(1.0 - class_present(pU)).mean()
        need_lower = F.relu(1.0 - class_present(pL)).mean()
        need_digit = F.relu(1.0 - class_present(pD)).mean()
        need_spec  = F.relu(1.0 - class_present(pS)).mean()
        class_loss = need_upper + need_lower + need_digit + need_spec

       
        init_soft = F.softmax(z_init, dim=-1).detach()  # [B, T, V]
        change_penalty = (y_probs - init_soft).abs().sum() / batch

        # Optional entropy constraint
        H_bits = entropy_bits_per_string(y_probs)
        entropy_loss = F.relu(H_min_bits - H_bits).mean()

        # Final loss
        loss = class_loss + entropy_loss + 0.1 * change_penalty

        opt.zero_grad()
        loss.backward()
        opt.step()

    return decode(y_probs.detach())



In [17]:


def generate_passwords23(batch=16, steps=200, T=12, H_min_bits=3.0, seed=0):
    torch.manual_seed(seed)

    model = TinyDecoder(T=T)
    for p in model.parameters():
        p.requires_grad = False                              # freeze decoder (we only optimize z)

    ## z = torch.randn(batch, 70, requires_grad=True)
    z = torch.randn(batch, T, V, requires_grad=True)

    
    opt = torch.optim.Adam([z], lr=0.05)
    y_probs = None

    for t in range(steps):
        tau = max(0.8 - 0.003 * t, 0.2)

        logits = z     ## model(z)                                    # [B, T, V]
        
        y_probs = st_gumbel_softmax(logits, tau)             # [B, T, V]

        pU = (y_probs * MASK_UPPER).sum(dim=-1)              # [B, T]
        pL = (y_probs * MASK_LOWER).sum(dim=-1)
        pD = (y_probs * MASK_DIGIT).sum(dim=-1)
        pS = (y_probs * MASK_SPEC ).sum(dim=-1)

        need_upper = F.relu(1.0 - class_present(pU)).mean()
        need_lower = F.relu(1.0 - class_present(pL)).mean()
        need_digit = F.relu(1.0 - class_present(pD)).mean()
        need_spec  = F.relu(1.0 - class_present(pS)).mean()
        
        class_loss = need_upper + need_lower + need_digit + need_spec

        H_bits       = entropy_bits_per_string(  y_probs  )            # [B]
        entropy_loss = F.relu(H_min_bits - H_bits).mean()

        loss = class_loss + entropy_loss

        opt.zero_grad()
        loss.backward()
        opt.step()

    return decode(  y_probs.detach()  )



In [18]:


def evaluate_passwords(password_list):
    """
    Takes a list of passwords and returns:
    - average score (0 to 4)
    - score distribution
    - crack time estimates
    """
    scores = []
    crack_times = []

    for pw in password_list:
        result = zxcvbn(pw)
        scores.append(result['score'])
        crack_times.append(result['crack_times_seconds']['offline_fast_hashing_1e10_per_second'])

    avg_score = sum(scores) / len(scores)
    score_distribution = {
        score: scores.count(score) for score in range(5)
    }

    return {
        'average_score': avg_score,
        'score_distribution': score_distribution,
        'avg_crack_time_secs': sum(crack_times) / len(crack_times)
    }



## More mask constraints 


In [19]:

num_passwords = 1024


In [20]:

constraintsGeneratedPasswords = generate_passwords(batch=num_passwords, steps=2000, T=12, H_min_bits=3.0)

## print("Sample:", constraintsGeneratedPasswords )



In [21]:

i = 0
for pass_generated in constraintsGeneratedPasswords[:12]:
    print(pass_generated)
    


!!aa0!00!!!!
0000aa!!!!!!
00a0000!!!!!
!0a0a!!!!!!!
0000!a0!!!!!
!!!000bbbbbb
00!a00!a!!!!
!a0aa0000000
!a!!!000bbbb
ba!000bbbbbb
0!000a!!!!!!
a0aa!!!!!!!!


In [22]:

result = evaluate_passwords(constraintsGeneratedPasswords)


In [23]:


print("Average score:", result['average_score'])
print("Score distribution:", result['score_distribution'])
print("Average crack time (seconds):", result['avg_crack_time_secs'])



Average score: 0.0419921875
Score distribution: {0: 1008, 1: 0, 2: 6, 3: 9, 4: 1}
Average crack time (seconds): 0.00122523792578125


In [24]:

pw = constraintsGeneratedPasswords

print(score_with_password_strength(pw))
print(average_entropy(pw))
print(entropy_stats(pw))


{'average_strength': 0.003410087888529521, 'weak%': 100.0, 'strong%': 0.0}
60.20389928273444
{'avg_entropy_bits': 60.20389928273444, 'min': 60.0, 'max': 73.04955409500407}
